In [30]:
import pandas as pd
import csv
import os
import re

In [24]:
DIR = os.path.join(os.path.expanduser("~"),'dataSet/audio/agender_distribution/')

In [25]:
def csv_list(DIR, files):
    raw_audio_list = [(DIR + files[j]) for j in range(len(files))]
    csv_list = [(DIR + (re.sub(r'.raw', '.mfc.csv', files[i]))) for i in range(len(files))]
    return csv_list

In [33]:
def ReadCSV(file):
    fin = file
    fout = pd.read_csv(fin, header=None)
    return fout

In [26]:
def NumberVectors(file_list):
    min_sample = 0
    c = 0
    list_vec_qtd = []
    for i in range(len(file_list)):
        file = file_list[i]
        df = ReadCSV(file)
        c += 1
        num_vectors = len(df.index)
        num_samples = num_vectors
        list_vec_qtd.append(num_vectors)
        if min_sample == 0:
            min_sample = num_samples
        elif num_samples < min_sample:
            min_sample = num_samples
            min_file = file_list[i]
        else:
            None
    return (min_sample, c, list_vec_qtd)

In [27]:
train_file_csv_full = 'trainSampleList_train.csv'
devel_file_csv_full = 'trainSampleList_devel.csv'
train_file_df_full = pd.read_csv(str(DIR + train_file_csv_full), delimiter=',', header=0)
devel_file_df_full = pd.read_csv(str(DIR + devel_file_csv_full), delimiter=',', header=0)

In [28]:
train_file_txt = 'trainSampleList_train.txt'
devel_file_txt = 'trainSampleList_devel.txt'
train_file_df = pd.read_table(str(DIR + train_file_txt), delimiter=' ', header=None)
train_file_raw = train_file_df[0]
devel_file_df = pd.read_table(str(DIR + devel_file_txt), delimiter=' ', header=None)
devel_file_raw = devel_file_df[0]

In [31]:
csv_list_train = csv_list(DIR, train_file_raw)
csv_list_dev = csv_list(DIR, devel_file_raw)

In [34]:
min_sample_train, counter_check_train, list_vec_qtd_train = NumberVectors(csv_list_train)
min_sample_dev, counter_check_dev, list_vec_qtd_dev = NumberVectors(csv_list_dev)
min_sample_dev, counter_check_dev, min_sample_train, counter_check_train

(97, 20549)

In [35]:
SampleListDataFrame_train = pd.DataFrame(list(zip(csv_list_train, list_vec_qtd_train)), columns=['Train Files', 'Qtd. of Vectors'])
SampleListDataFrame_dev = pd.DataFrame(list(zip(csv_list_dev, list_vec_qtd_dev)), columns=['Dev Files', 'Qtd. of Vectors'])
counter_train = SampleListDataFrame_train['Qtd. of Vectors'].value_counts()
counter_train = counter_train.to_frame('qtd')
counter_dev = SampleListDataFrame_dev['Qtd. of Vectors'].value_counts()
counter_dev = counter_dev.to_frame('qtd')

In [36]:
threshold_train = 169
files_to_use_train = 0
for k in range(len(counter_train)):
    row_index = counter_train.index[k]
    row_value = counter_train.iat[k, 0]
    if row_index >= threshold_train:
        files_to_use_train = files_to_use_train + row_value
files_to_use_train

28430

In [37]:
threshold_dev = 169
files_to_use_dev = 0
for k in range(len(counter_dev)):
    row_index = counter_dev.index[k]
    row_value = counter_dev.iat[k, 0]
    if row_index >= threshold_dev:
        files_to_use_dev = files_to_use_dev + row_value
files_to_use_dev

17939

In [89]:
train_csv_use = []
train_not_used = []
for i in range(len(SampleListDataFrame_train.index)):
    train_vectors = SampleListDataFrame_train.iat[i,1]
    if train_vectors >= threshold_train:
        train_csv_use.append(SampleListDataFrame_train.iat[i,0])
    else:
        train_not_used.append(SampleListDataFrame_train.iat[i,0])

In [91]:
dev_csv_use = []
dev_not_used = []
for i in range(len(SampleListDataFrame_dev.index)):
    dev_vectors = SampleListDataFrame_dev.iat[i,1]
    if dev_vectors >= threshold_dev:
        dev_csv_use.append(SampleListDataFrame_dev.iat[i,0])
    else:
        dev_not_used.append(SampleListDataFrame_dev.iat[i,0])

In [92]:
for i in range(len(train_csv_use)):
    regex1 = '/home/arielferreira/dataSet/audio/agender_distribution/'
    regex2 = '.mfc.csv'
    train_csv_use[i] = re.sub(regex1, '', train_csv_use[i])
    train_csv_use[i] = re.sub(regex2, '.wav', train_csv_use[i])

In [93]:
for i in range(len(dev_csv_use)):
    regex = '/home/arielferreira/dataSet/audio/agender_distribution/'
    regex2 = '.mfc.csv'
    dev_csv_use[i] = re.sub(regex1, '', dev_csv_use[i])
    dev_csv_use[i] = re.sub(regex2, '.wav', dev_csv_use[i])

In [95]:
train_csv_use_df = pd.DataFrame(train_csv_use).rename(columns={0: 'mfc_files'})
dev_csv_use_df = pd.DataFrame(dev_csv_use).rename(columns={0: 'mfc_files'})

In [104]:
train_file_df_full

,file,class
0,wav_traindevel/1006/1/a11006s14.wav,1
1,wav_traindevel/1006/1/a11006s15.wav,1
2,wav_traindevel/1006/1/a11006s16.wav,1
3,wav_traindevel/1006/1/a11006s17.wav,1
4,wav_traindevel/1006/1/a11006s18.wav,1
...,...,...
32522,wav_traindevel/7994/6/a17994s5.wav,7
32523,wav_traindevel/7994/6/a17994s6.wav,7
32524,wav_traindevel/7994/6/a17994s7.wav,7
32525,wav_traindevel/7994/6/a17994s8.wav,7


In [97]:
train_csv_use_df    

,mfc_files
0,wav_traindevel/1006/1/a11006s14.wav
1,wav_traindevel/1006/1/a11006s16.wav
2,wav_traindevel/1006/2/a11006s11.wav
3,wav_traindevel/1006/2/a11006s12.wav
4,wav_traindevel/1006/2/a11006s13.wav
...,...
28425,wav_traindevel/7994/6/a17994s5.wav
28426,wav_traindevel/7994/6/a17994s6.wav
28427,wav_traindevel/7994/6/a17994s7.wav
28428,wav_traindevel/7994/6/a17994s8.wav


In [108]:
train_audio = []
train_label = []
for j in range(len(train_csv_use_df.index)):
    row = train_csv_use_df.iat[j, 0]
    for i in range(len(train_file_df_full.index)):
        if row == train_file_df_full.iat[i, 0]:
            train_audio.append(train_file_df_full.iat[i, 0])
            train_label.append(train_file_df_full.iat[i, 1])
        else:
            None

In [110]:
len(train_audio), len(train_label)

(28430, 28430)

In [111]:
devel_audio = []
devel_label = []
for j in range(len(dev_csv_use_df.index)):
    row = dev_csv_use_df.iat[j, 0]
    for i in range(len(devel_file_df_full.index)):
        if row == devel_file_df_full.iat[i, 0]:
            devel_audio.append(devel_file_df_full.iat[i, 0])
            devel_label.append(devel_file_df_full.iat[i, 1])
        else:
            None